In [1]:
import pathlib
import random
import string
import re
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization



In [2]:


with open('qed_new.txt', 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
    
text_pairs=[]
for line in lines[: min(len(lines), len(lines)-1)]:
    input_exp, target_exp = line.split('\t')
    
    target_exp = "[start] " + target_exp + " [end]"
    text_pairs.append((input_exp,target_exp ))

random.shuffle(text_pairs)
text_pairs = text_pairs[:10000]

In [3]:
text_pairs1 = []

for i in range(len(text_pairs)):
    if len(text_pairs[i][1]) < 2000:
        text_pairs1.append(text_pairs[i])
        
text_pairs = text_pairs1
print(len(text_pairs))

10000


In [4]:

num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

10000 total pairs
7000 training pairs
1500 validation pairs
1500 test pairs


In [5]:
train_input_texts = [pair[0] for pair in train_pairs]
train_output_texts = [pair[1] for pair in train_pairs]
train_input_texts[1]

'4/27*i*e^3*(p_2_%\\mu*gamma_{+%\\mu,%eta,%eta}*gamma_{%\\sigma,%eta,%eta}*gamma_{+%\\sigma,%eta,%beta}*gamma_{%\\sigma,%eta,%gam}*A_{j,+%\\sigma}(p_2)*c_{i,%eta}(p_5)^(*)*c_{l,%gam}(p_4)*s_{k,%beta}(p_3)*s_{i,%eta}(p_1)^(*) + (-2)*p_5_%\\sigma*gamma_{%\\sigma,%eta,%gam}*gamma_{+%\\sigma,%eta,%beta}*A_{j,+%\\sigma}(p_2)*c_{i,%eta}(p_5)^(*)*c_{l,%gam}(p_4)*s_{k,%beta}(p_3)*s_{i,%eta}(p_1)^(*))/((s_22 + (-2)*s_25)*(s_11 + (-2)*s_13 + s_33))'

In [6]:
def separ_sq(data):
    
    for r in (('*', ' * '), ('/', ' / '), ('^', ' ^ '), ('(', '( '), (')', ' )')):
        data = data.replace(*r) 
        
    return data

train_output_texts = [separ_sq(train_output_texts[i]) for i in range(len(train_output_texts))]
train_output_texts[0]

'[start] -256 * e ^ 6 * ( m_b ^ 2 * ( 4 * m_u ^ 4 + 4 * m_u ^ 2 * s_34 + 3 * m_u ^ 2 * s_35 + 4 * m_u ^ 2 * s_44 + 4 * m_u ^ 2 * s_45 - 2 * s_34 * s_45 - 2 * s_34 * s_55 + s_35 * s_44 - s_35 * s_55 ) + 2 * m_u ^ 4 * s_12 + 2 * m_u ^ 2 * s_12 * s_44 + 2 * m_u ^ 2 * s_12 * s_45 + 4 * m_u ^ 2 * s_13 * s_24 + 3 * m_u ^ 2 * s_13 * s_25 + 4 * m_u ^ 2 * s_14 * s_23 + 3 * m_u ^ 2 * s_15 * s_23 - 2 * s_13 * s_24 * s_45 - 2 * s_13 * s_24 * s_55 + s_13 * s_25 * s_44 - s_13 * s_25 * s_55 - 2 * s_14 * s_23 * s_45 - 2 * s_14 * s_23 * s_55 + s_15 * s_23 * s_44 - s_15 * s_23 * s_55 ) / ( 729 * ( s_11 + 2 * s_12 + s_22 ) ^ 2 * ( -m_u ^ 2 + s_44 + 2 * s_45 + s_55 ) ^ 2 ) [end]'

In [7]:
def separ_amp(data):
    
    for r in (('*', ' * '),  ('  ', ' ')) :
        data = data.replace(*r) 
        
    return data


train_input_texts = [separ_amp(train_input_texts[i]) for i in range(len(train_input_texts))]
train_input_texts[0]

'4/27 * i * e^3 * (p_4_%\\lambda * gamma_{+%\\lambda,%eta,%eta} * gamma_{%\\tau,%eta,%eta} * gamma_{+%\\tau,%gam,%del} * gamma_{%\\tau,%eta,%eps} * A_{i,+%\\tau}(p_4)^( * ) * b_{j,%gam}(p_1)^( * ) * b_{k,%del}(p_2) * u_{l,%eps}(p_3) * u_{j,%eta}(p_5)^( * ) + p_5_%\\lambda * gamma_{+%\\lambda,%eta,%eta} * gamma_{%\\tau,%eta,%eta} * gamma_{+%\\tau,%gam,%del} * gamma_{%\\tau,%eta,%eps} * A_{i,+%\\tau}(p_4)^( * ) * b_{j,%gam}(p_1)^( * ) * b_{k,%del}(p_2) * u_{l,%eps}(p_3) * u_{j,%eta}(p_5)^( * ) + m_u * gamma_{+%\\lambda,%eta,%eta} * gamma_{+%\\tau,%gam,%del} * gamma_{%\\tau,%eta,%eps} * A_{i,%\\lambda}(p_4)^( * ) * b_{j,%gam}(p_1)^( * ) * b_{k,%del}(p_2) * u_{l,%eps}(p_3) * u_{j,%eta}(p_5)^( * ))/((s_11 + 2 * s_12 + s_22) * (m_u^2 + -s_44 + (-2) * s_45 + -s_55))'

In [ ]:
train_output_texts[108]

In [ ]:
train_input_texts[0]

In [11]:
#normal





vocab_size = 5000
sequence_length = 100
batch_size = 64



input_vectorization = TextVectorization(
    max_tokens=vocab_size, 
    output_mode="int", output_sequence_length=sequence_length, standardize=None)

output_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1, standardize=None)



input_vectorization.adapt(train_input_texts)
output_vectorization.adapt(train_output_texts)


target_tokens = output_vectorization.get_vocabulary()
input_tokens = input_vectorization.get_vocabulary()




print('number of input tokens: ', len(input_tokens))
print('number of target tokens: ', len(target_tokens))

number of input tokens:  4241
number of target tokens:  93


In [9]:
VOCAB_SIZE = vocab_size
MAX_SAMPLES = 500
BUFFER_SIZE = 20000
MAX_LENGTH = sequence_length
EMBED_DIM = 512
LATENT_DIM = 512
NUM_HEADS = 8
BATCH_SIZE = batch_size



In [12]:
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

train_input_texts = [pair[0] for pair in train_pairs]
train_output_texts = [pair[1] for pair in train_pairs]

In [13]:

def format_dataset(input_exp, target_exp):
    input_exp = input_vectorization(input_exp)
    target_exp = output_vectorization(target_exp)
    return ({"encoder_inputs": input_exp, "decoder_inputs": target_exp[:, :-1],}, target_exp[:, 1:])


def make_dataset(pairs):
    ampl_texts, sqampl_texts = zip(*pairs)
    ampl_texts = list(ampl_texts)
    sqampl_texts = list(sqampl_texts)
    dataset = tf.data.Dataset.from_tensor_slices((ampl_texts, sqampl_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.shuffle(2048).prefetch(16).cache()



train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [14]:
# class TransformerEncoder(layers.Layer):
#     def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
#         super(TransformerEncoder, self).__init__(**kwargs)
#         self.embed_dim = embed_dim
#         self.dense_dim = dense_dim
#         self.num_heads = num_heads
#         self.attention = layers.MultiHeadAttention(
#             num_heads=num_heads, key_dim=embed_dim
#         )
#         self.dense_proj = keras.Sequential(
#             [layers.Dense(dense_dim, activation="relu"), layers.Dense(embed_dim),]
#         )
#         self.layernorm_1 = layers.LayerNormalization()
#         self.layernorm_2 = layers.LayerNormalization()
#         self.supports_masking = True

#     def call(self, inputs, mask=None):
#         if mask is not None:
#             padding_mask = tf.cast(mask[:, tf.newaxis, tf.newaxis, :], dtype="int64")
#         attention_output = self.attention(
#             query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
#         )
#         proj_input = self.layernorm_1(inputs + attention_output)
#         proj_output = self.dense_proj(proj_input)
#         return self.layernorm_2(proj_input + proj_output)


class FNetEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, **kwargs):
        super(FNetEncoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(dense_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs):
        # Casting the inputs to complex64
        inp_complex = tf.cast(inputs, tf.complex64)
        # Projecting the inputs to the frequency domain using FFT2D and
        # extracting the real part of the output
        fft = tf.math.real(tf.signal.fft2d(inp_complex))
        proj_input = self.layernorm_1(inputs + fft)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)



class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super(PositionalEmbedding, self).__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)


class FNetDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super(FNetDecoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(latent_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        return tf.tile(mask, mult)


def create_model():
    encoder_inputs = keras.Input(shape=(None,), dtype="int32", name="encoder_inputs")
    x = PositionalEmbedding(MAX_LENGTH, VOCAB_SIZE, EMBED_DIM)(encoder_inputs)
    encoder_outputs = FNetEncoder(EMBED_DIM, LATENT_DIM)(x)
    encoder = keras.Model(encoder_inputs, encoder_outputs)
    decoder_inputs = keras.Input(shape=(None,), dtype="int32", name="decoder_inputs")
    encoded_seq_inputs = keras.Input(
        shape=(None, EMBED_DIM), name="decoder_state_inputs"
    )
    x = PositionalEmbedding(MAX_LENGTH, VOCAB_SIZE, EMBED_DIM)(decoder_inputs)
    x = FNetDecoder(EMBED_DIM, LATENT_DIM, NUM_HEADS)(x, encoded_seq_inputs)
    x = layers.Dropout(0.5)(x)
    decoder_outputs = layers.Dense(VOCAB_SIZE, activation="softmax")(x)
    decoder = keras.Model(
        [decoder_inputs, encoded_seq_inputs], decoder_outputs, name="outputs"
    )
    decoder_outputs = decoder([decoder_inputs, encoder_outputs])
    fnet = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs, name="fnet")
    return fnet



# class PositionalEmbedding(layers.Layer):
#     def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
#         super(PositionalEmbedding, self).__init__(**kwargs)
#         self.token_embeddings = layers.Embedding(
#             input_dim=vocab_size, output_dim=embed_dim
#         )
#         self.position_embeddings = layers.Embedding(
#             input_dim=sequence_length, output_dim=embed_dim
#         )
#         self.sequence_length = sequence_length
#         self.vocab_size = vocab_size
#         self.embed_dim = embed_dim

#     def call(self, inputs):
#         length = tf.shape(inputs)[-1]
#         positions = tf.range(start=0, limit=length, delta=1)
#         embedded_tokens = self.token_embeddings(inputs)
#         embedded_positions = self.position_embeddings(positions)
#         return embedded_tokens + embedded_positions

#     def compute_mask(self, inputs, mask=None):
#         return tf.math.not_equal(inputs, 0)


# class TransformerDecoder(layers.Layer):
#     def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
#         super(TransformerDecoder, self).__init__(**kwargs)
#         self.embed_dim = embed_dim
#         self.latent_dim = latent_dim
#         self.num_heads = num_heads
#         self.attention_1 = layers.MultiHeadAttention(
#             num_heads=num_heads, key_dim=embed_dim
#         )
#         self.attention_2 = layers.MultiHeadAttention(
#             num_heads=num_heads, key_dim=embed_dim
#         )
#         self.dense_proj = keras.Sequential(
#             [layers.Dense(latent_dim, activation="relu"), layers.Dense(embed_dim),]
#         )
#         self.layernorm_1 = layers.LayerNormalization()
#         self.layernorm_2 = layers.LayerNormalization()
#         self.layernorm_3 = layers.LayerNormalization()
#         self.supports_masking = True

#     def call(self, inputs, encoder_outputs, mask=None):
#         causal_mask = self.get_causal_attention_mask(inputs)
#         if mask is not None:
#             padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int64")
#             padding_mask = tf.minimum(padding_mask, causal_mask)

#         attention_output_1 = self.attention_1(
#             query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
#         )
#         out_1 = self.layernorm_1(inputs + attention_output_1)

#         attention_output_2 = self.attention_2(
#             query=out_1,
#             value=encoder_outputs,
#             key=encoder_outputs,
#             attention_mask=padding_mask,
#         )
#         out_2 = self.layernorm_2(out_1 + attention_output_2)

#         proj_output = self.dense_proj(out_2)
#         return self.layernorm_3(out_2 + proj_output)

#     def get_causal_attention_mask(self, inputs):
#         input_shape = tf.shape(inputs)
#         batch_size, sequence_length = input_shape[0], input_shape[1]
#         i = tf.range(sequence_length)[:, tf.newaxis]
#         j = tf.range(sequence_length)
#         mask = tf.cast(i >= j, dtype="int64")
#         mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
#         mult = tf.concat(
#             [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
#             axis=0,
#         )
#         return tf.tile(mask, mult)

In [15]:

fnet = create_model()
fnet.compile("adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

fnet.fit(train_ds, epochs=10, validation_data=val_ds)


Epoch 1/10
  5/110 [>.............................] - ETA: 27:18 - loss: 2.2640 - accuracy: 0.4015 

KeyboardInterrupt: 

In [ ]:
VOCAB = vectorizer.get_vocabulary()


def decode_sentence(input_sentence):
    # Mapping the input sentence to tokens and adding start and end tokens
    tokenized_input_sentence = vectorizer(
        tf.constant("[start] " + preprocess_text(input_sentence) + " [end]")
    )
    # Initializing the initial sentence consisting of only the start token.
    tokenized_target_sentence = tf.expand_dims(VOCAB.index("[start]"), 0)
    decoded_sentence = ""

    for i in range(MAX_LENGTH):
        # Get the predictions
        predictions = fnet.predict(
            {
                "encoder_inputs": tf.expand_dims(tokenized_input_sentence, 0),
                "decoder_inputs": tf.expand_dims(
                    tf.pad(
                        tokenized_target_sentence,
                        [[0, MAX_LENGTH - tf.shape(tokenized_target_sentence)[0]]],
                    ),
                    0,
                ),
            }
        )
        # Calculating the token with maximum probability and getting the corresponding word
        sampled_token_index = tf.argmax(predictions[0, i, :])
        sampled_token = VOCAB[sampled_token_index.numpy()]
        # If sampled token is the end token then stop generating and return the sentence
        if tf.equal(sampled_token_index, VOCAB.index("[end]")):
            break
        decoded_sentence += sampled_token + " "
        tokenized_target_sentence = tf.concat(
            [tokenized_target_sentence, [sampled_token_index]], 0
        )

    return decoded_sentence
test_input_texts = [pair[0] for pair in test_pairs]
test_output_texts = [pair[1] for pair in test_pairs]

decoded_sentence(test_input_texts[0])

In [ ]:
target_tokens = output_vectorization.get_vocabulary()
target_index_lookup = dict(zip(range(len(target_tokens)), target_tokens))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = input_vectorization([input_sentence])
       
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = output_vectorization([decoded_sentence])[:, :-1]
        
        predictions = fnet([tokenized_input_sentence, tokenized_target_sentence])

        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = target_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break
    return decoded_sentence


test_input_texts = [pair[0] for pair in test_pairs]
test_output_texts = [pair[1] for pair in test_pairs]

listt = []
for i in random.sample(range(0,len(test_input_texts)), 10):
    input_sentence = test_input_texts[i]
    translated = decode_sequence(input_sentence)
    print('Actual:    ', test_output_texts[i], '\n')
    print('Predicted: ', translated, '\n')
    print(test_output_texts[i]==translated, '\n')
    listt.append(test_output_texts[i]==translated)
    
print(listt.count(True) , 'is True out of' , len(listt))

In [ ]:
def compare_tokens(a,p):
    res = sum(x == y for x, y in zip(a.split(), p.split()))
    prec = res/len(a.split())
    return prec

listt = []
list_f = []
for i in random.sample(range(0,len(test_input_texts)), 10):
    input_sentence = test_input_texts[i]
    translated = decode_sequence(input_sentence)
    print('Actual:    ', test_output_texts[i], '\n')
    print('Predicted: ', translated, '\n')
    print(test_output_texts[i]==translated, '\n')
    listt.append(test_output_texts[i]==translated)
    x = test_output_texts[i]==translated
    if  x == False:
        print(compare_tokens(test_output_texts[i],translated), '\n')
        list_f.append(compare_tokens(test_output_texts[i], translated))
    
print(listt.count(True) , 'is True out of' , len(listt), '\n')
print('precentage of true sequences:  ' , listt.count(True)/len(listt), '\n')
print(list_f, '\n')
print('precentage of true tokens:  ', sum(list_f)/len(list_f))

In [ ]:
a = [1,3,5]
b = [1,4]
x = a[0]==b[1]
if x is False:
    print('y')

In [ ]:
def separ_amp(data):
    
    for r in (('*', ' * '), ('( * )', '(*)'), ('}', '} '), ('  ', ' ')) :
        data = data.replace(*r) 
        
    return data




In [ ]:
import sympy as sp


def nu_evaluation(data, e_, m_e_, m_mu_, m_t_, m_u_, m_d_, m_s_, m_c_, m_tt_, m_b_, s_11_, s_12_, s_13_, s_14_, s_15_, s_22_, s_23_, s_24_, s_25_, s_33_, s_34_, s_35_, s_44_,s_45_ ,s_55_):
    
    e, i, m_e, m_mu, m_t, m_u, m_d, m_s, m_c, m_tt, m_b, s_11, s_12, s_13, s_14, s_15, s_22, s_23, s_24, s_25, s_33, s_34, s_35, s_44,s_45 ,s_55 = sp.symbols('e, i, m_e, m_mu, m_t, m_u, m_d, m_s, m_c, m_tt, m_b, s_11, s_12, s_13, s_14, s_15, s_22, s_23, s_24, s_25, s_33, s_34, s_35, s_44,s_45 ,s_55')
    expr = data.replace("^", "**")
    expr = sp.core.sympify(expr)
    result = expr.subs([(e,e_),(m_e,m_e_ ),(m_mu, m_mu_), (m_t, m_t_), (m_u, m_u_), (m_d, m_d_), 
                        (m_s, m_s_), (m_c, m_c_), (m_tt, m_tt_), (m_b, m_b_),(i, sp.sqrt(-1)), (s_11, s_11_), (s_12, s_12_),(s_13, s_13_), (s_14, s_14_),
                          (s_15, s_15_), (s_22, s_22_) ,(s_23, s_23_), (s_24, s_24_), (s_25, s_25_), (s_33, s_33_) ,(s_34, s_34_), (s_35, s_35_), (s_44, s_44_), (s_45, s_45_), (s_55,s_55_)])
    result = str(result).replace("**", '^')
    
    return result
    

In [ ]:
data = 'e6^ m_t2^ * -128m_mu2^ * s_14* 128m_mu2^ * s_44* - 64s_14* s_35* - 256s_34* s_45*-* 16s_122^ * m_mu2^ 2s_35* + s_55+2^*/ e6^ m_t* -32m_mu2^ * m_t* s_13* 32m_mu2^ * m_t* s_15* - 32m_mu2^ * m_t* s_34* - 32m_mu2^ * m_t* s_45* - 16m_t* s_13* s_35* - 16m_t* s_15* s_35* - m_t32m_mu2^ * s_45* 32s_34* s_35*+* - m_t32s_34* s_55* 32s_35* s_45*+*-* 2s_122^ * m_mu2^ 2s_35* + s_55+2^*/ + e6^ 64m_mu4^ * m_t2^ * 32m_mu2^ * m_t2^ * s_35* + 64m_mu2^ * m_t2^ * s_44* + 64m_mu2^ * m_t2^ * s_55* + 16m_mu2^ * s_13* s_45* + 16m_mu2^ * s_14* s_44* + 16m_mu2^ * s_15* s_34* - m_mu2^ -64m_mu2^ * s_14* 128s_13* s_34*+* 4/ + m_mu2^ -64s_14* s_55* 128s_15* s_45*+* 4/ + 32m_t2^ * s_35* s_44* + 32m_t2^ * s_35* s_55* + 32s_13* s_34* s_35* + 16s_13* s_44* s_45* - 16s_13* s_45* s_55* - 64s_14* s_34* s_45* + 16s_15* s_34* s_44* - 16s_15* s_34* s_55* + 32s_15* s_35* s_45*+* 4s_122^ * m_mu2^ 2s_35* + s_55+2^*/ + e6^ 64m_mu2^ * m_t2^ * s_34* 64m_mu2^ * m_t2^ * s_35* + 64m_mu2^ * m_t2^ * s_45* + 16m_mu2^ * s_13* s_44* + 16m_mu2^ * s_14* s_45* + 16m_mu2^ * s_15* s_44* + 16m_mu2^ * s_15* s_45* + m_mu2^ 64s_13* s_45* 64s_14* s_35* - 64s_15* s_34*+* 4/ + 32m_t2^ * s_34* s_35* + 32m_t2^ * s_352^ * + 32m_t2^ * s_35* s_45* + 16s_13* s_34* s_45* + 16s_13* s_34* s_55* + 16s_13* s_452^ * - 16s_14* s_34* s_35* + 16s_14* s_34* s_55* + 16s_14* s_35* s_45* + 16s_15* s_342^ * - 16s_15* s_34* s_45*+* 2s_122^ * m_mu2^ 2s_35* + s_55+2^*/ + e3^ i* m_t* e3^ i* m_t* -128m_mu2^ * s_14* 128m_mu2^ * s_44* - 64s_14* s_35* - 256s_34* s_45*-* 4s_12* m_mu2^ 2s_35* + s_55+*/ e3^ i* m_t* 64m_mu2^ * m_t2^ * 16m_mu2^ * s_14* + 32m_t2^ * s_35* + 16s_13* s_45* + 16s_15* s_34*+* s_12m_mu2^ 2s_35* + s_55+*/+* 4s_12* m_mu2^ 2s_35* + s_55+*/-'

In [ ]:
ampl = separ_amp('-1/2*i*e^3*(p_3_%\rho*gamma_{+%\rho,%eps,%del}*gamma_{%\rho,%eta,%del}*A_{j,+%\rho}(p_5)^(*)*e_{i,%del}(p_1)*e_{k,%del}(p_2)*e_{l,%eps}(p_3)^(*)*e_{i,%eta}(p_4)^(*) + 1/2*p_5_%\sigma*gamma_{%\rho,%eps,%eps}*gamma_{+%\rho,%eps,%del}*gamma_{%\rho,%eta,%del}*gamma_{+%\sigma,%eps,%eps}*A_{j,+%\rho}(p_5)^(*)*e_{i,%del}(p_1)*e_{k,%del}(p_2)*e_{l,%eps}(p_3)^(*)*e_{i,%eta}(p_4)^(*))/((m_e^2 + -s_24)*s_35)')
ampl

In [ ]:
translated = decode_sequence(ampl)
translated